In [1]:
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

sns.set_style('darkgrid')

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
sns.get_dataset_names()

In [ ]:
df = sns.load_dataset('titanic')

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
plt.figure(figsize=(10, 5));
survived = df.survived.value_counts()
survived.index = [survived]
sns.barplot(survived.index);

In [ ]:
plt.figure(figsize=(10, 5))
df.embarked.value_counts().plot.barh();
plt.title('Embarked');

In [ ]:
plt.figure(figsize=(10, 5))
df['class'].value_counts().plot.barh();
plt.title('Class');

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x='alive', data=df, hue='sex');

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(x='age', data=df, bins=20);
plt.title('Age');

In [ ]:
plt.figure(figsize=(10, 5))
df.age.value_counts().nlargest(11).plot.barh();
plt.xlabel('Count of Age')
plt.ylabel('Age');

In [ ]:
df.corr().iloc[0, 1]

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df.corr(), annot=True);

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x='alone', data=df, hue='sex');

In [ ]:
df['embarked'].fillna(value=df['embarked'].mode()[0], inplace=True)
df['embark_town'].fillna(value=df['embark_town'].mode()[0], inplace=True)

In [ ]:
def get_label_class(val):
    if val == 'First':
        val = 0
    elif val == 'Second':
        val = 1
    elif val == 'Third':
        val = 2
    return val

def get_label_embarked(val):
    if val == 'S':
        val = 0
    elif val == 'C':
        val = 1
    elif val == 'Q':
        val = 2
    return val

def get_label_embarktown(val):
    if val == 'Southampton':
        val = 0
    elif val == 'Cherbourg':
        val = 1
    elif val == 'Queenstown':
        val = 2
    return val

In [ ]:
df['sex'] = np.where(df['sex'] == 'male', 1, 0)

df['class'] = df['class'].map(get_label_class)

df.drop(['who'], axis=1, inplace=True)

df['embarked'] = df['embarked'].map(get_label_embarked)

df['adult_male'] = np.where(df['adult_male'] == True, 1, 0)

df['embark_town'] = df['embark_town'].map(get_label_embarktown)

df.drop(['deck'], axis=1, inplace=True)

df.drop(['alive'], axis=1, inplace=True)

df['alone'] = np.where(df['alone'] == True, 1, 0)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x='age', data=df);
plt.title('Age');

In [ ]:
upper_value = df['age'].mean() + (3 * df.age.std())
lower_value = df['age'].mean() - (3 * df.age.std())

print(upper_value)
print(lower_value)

In [ ]:
df.loc[df.age > upper_value, 'age'] = upper_value

In [ ]:
sns.boxplot(x='age', data=df);

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df.corr(), annot=True);

In [ ]:
df.drop(['adult_male', 'embarked'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(df.fare);

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x='fare', data=df);
plt.title('Fare')

In [ ]:
IQR = df.fare.quantile(0.75) - df.fare.quantile(0.25)

lower_value = df.fare.quantile(0.25) - (IQR * 1.5)
upper_value =  df.fare.quantile(0.75) + (IQR * 1.5)

# lower_value_extreme = df.fare.quantile
print(lower_value)
print(upper_value)

In [ ]:
df.loc[df.fare > upper_value, 'fare'] = upper_value

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(df.fare);

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(x='fare', data=df);

In [ ]:
df.age = df.age.fillna(value=df.age.mean())

In [ ]:
df.isna().sum()

In [ ]:
X = df.loc[:, 'pclass':]
y = df.loc[:, 'survived']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X = X.values
y = y.values

In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=9))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(units=64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(units=32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(
    patience=5,
    monitor='val_loss',
)

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, callbacks=[callback], validation_split=0.2, epochs=100, verbose=1)

In [ ]:
plt.figure(figsize=(10, 8));
plt.plot(history.history['loss'], label='train_loss');
plt.plot(history.history['val_loss'], label='val_loss');
plt.title('Loss Vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Val Loss']);

In [ ]:
plt.figure(figsize=(10, 10));
plt.plot(history.history['accuracy'], label='train_acc');
plt.plot(history.history['val_accuracy'], label='val_acc');
plt.title('Accuracy Vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Val Accuracy']);

In [ ]:
pred = model.predict(X_test)

In [ ]:
predic = []
for i in pred:
    if i > 0.5:
        predic.append(1)
    else:
        predic.append(0)

In [ ]:
predic = np.array(predic)

In [ ]:
print("----------------------------------------------------Accuracy------------------------------------------------------")
print(accuracy_score(y_test, predic))
print()

print("---------------------------------------------------Classification Report---------------------------------------------")
print(classification_report(y_test, predic))
print()

print("-------------------------------------------------Confustion Metrics----------------------------------------------------")
plt.figure(figsize=(10, 10));
sns.heatmap(confusion_matrix(y_test, predic), annot=True);

In [ ]:
x = pd.DataFrame({'Actual': y_test, 'Predicted': predic}, columns=['Actual', 'Predicted'])

In [ ]:
x.head(50)

In [ ]:
len(df.columns)